# MATH 210 Project I

## Solving Linear Programming Problems  with `scipy.optimize.linprog`

SciPy is one of the core scientific computing packages in Python and the subpackage `scipy.optimize` is a collection of functions that produce two types of non-trivial data of an optimization problem *(see the [documentation](https://docs.scipy.org/doc/scipy-0.18.1/reference/optimize.html#optimization))*:

1. Finding the optimal solution/s: 
${\hspace 1mm} \boldsymbol{f} := X:\{x_{1},x_{2},\ldots,x_{n}\} \longrightarrow \boldsymbol{R^{{\hspace 0.5mm}n}}\\
\exists{{\hspace 1mm}x^{*}}\in{X},{\hspace 0.5mm}\forall{{\hspace 1mm}x}\in{X}{\hspace 2.5mm}\text{such that:}
{\hspace 5mm}\boldsymbol{min}{\hspace 2.5mm}f(x^{*}) \leq f(x) {\hspace 4mm}{\text{or}} 
{\hspace 5mm}\boldsymbol{max}{\hspace 2.5mm}f(x^{*}) \geq f(x)$
2. Finding roots of different kinds of functions:
${\hspace 5mm}\exists{\hspace 0.75mm}x, {\hspace 1mm}\text{s.t.}{\hspace 2mm} f(x) = 0$

The **goal** of this notebook is to explore **the methodology** of an optimization function in the subpackage `scipy.optimize` that **solves linear programming problems**. The function `scipy.optimize.linprog`, which minimizes a linear objective function subjected to linear equality and inequality constraints *(see the [documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linprog.html))*, is a useful function in determining the optimal solution of different types of linear programming problems. By the end of this notebook, the reader will be able to apply this `scipy.optimize` function to the following types of L.P. problems:

* The Standard Maximum Problem
* The Standard Minimum Problem

## Contents

1. Linear Programming Model: `optimize.linprog`
   * Simplex Algorithm Method
   * Special Cases of L.P. Problems
   * Function Definition to Solve Feasible L.P Problems

2. The Standard Maximum (Minimum) Problem
3. Exercises

In [ ]:
from scipy.optimize import linprog
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## 1. Linear Programming Model

A [linear programming](http://web.cs.iastate.edu/~cs511/handout10/LP%20Basics.pdf) (L.P.) problem in [standard form](https://en.wikipedia.org/wiki/Linear_programming#Standard_form) is of the form:
$$
{\hspace 45mm}
\begin{alignat*}{3}
\textbf{"Primal"}& \quad && \quad& && \quad\quad\quad \textbf{"Dual"} \\
Maximum{\hspace 3.8mm}\bar{c}^{\intercal}{\hspace 0.5mm}\bar{x}& \quad && &{\|} \quad &&
Minimum{\hspace 3.8mm}\quad\bar{b}^{\intercal}{\hspace 0.5mm} &\bar{y} 
\quad && \\
\text{subject to}{\hspace 5mm}\boldsymbol{A}{\hspace 0.5mm}\bar{x}& \leq \bar{\boldsymbol{b}} \quad && &{\|} \quad &&
\text{subject to}{\hspace 5mm}\quad\boldsymbol{A}{\hspace 0.5mm} &\bar{y} \geq \bar{\boldsymbol{c}} \\
x &\geq 0 \quad && &{\|} \quad &&
&y\geq 0 
\end{alignat*}
$$
${\hspace 1mm}$
where,
$$
{\hspace -2mm}
A = 
\begin{pmatrix}
a_{11} & a_{12} & \cdots & a_{1n} \\
a_{21} & a_{22} & \cdots & a_{2n} \\
\vdots & \vdots & \ddots & \vdots \\
a_{m1} & a_{m2} & \cdots & a_{mn}
\end{pmatrix}
,{\hspace 5mm}
\bar{c} = 
\begin{pmatrix}
c_{1}\\c_{2} \\ \vdots \\c_{n}
\end{pmatrix}
,{\hspace 5mm}
\bar{b} = 
\begin{pmatrix}
b_{1}\\b_{2} \\ \vdots \\b_{m}
\end{pmatrix}
,{\hspace 5mm}
\bar{x} = 
\begin{pmatrix}
x_{1}\\x_{2} \\ \vdots \\x_{n}
\end{pmatrix},{\hspace 5mm}
\bar{y} = 
\begin{pmatrix}
y_{1}\\y_{2} \\ \vdots \\y_{n}
\end{pmatrix}
$$

### Simplex Algorithm Method
When the L.P. problem is in standard form, the optimal solution/s can easily be found through the [simplex algorithm method](https://en.wikipedia.org/wiki/Simplex_algorithm), which is an algorithm that perform iterations on pivots to search for a feasible solution of the linear problem *(see the [documentation](https://docs.scipy.org/doc/scipy/reference/optimize.linprog-simplex.html#optimize-linprog-simplex))*. Let us elaborate the process of this algorithm:

$$
\boldsymbol{\Lambda} = \{1,2,\ldots,n,n+1,\dots,n+m\}, {\hspace 15mm}
B \subseteq \Lambda, {\hspace 15mm}
N = \Lambda \backslash{\hspace 0.5mm}B{\hspace 0.5mm}, \\
\boldsymbol{\tilde{A}_{\Lambda}} = A{\hspace 1mm}{\|}{\hspace 1mm}I
$$

If the [Primal](https://en.wikipedia.org/wiki/Duality) L.P. problem is in standard form, then an initial *dictionary* can be formulated:

$$
\begin{align}
\boldsymbol{x_{B}} &= A_{B}^{-1}{\hspace 0.5mm} b - A_{B}^{-1}{\hspace 0.5mm}A_{N}{\hspace 0.5mm}x_{N}\\
\boldsymbol{z} &= c_{B}^{T} {\hspace 0.5mm} A_{B}^{-1} {\hspace 0.5mm} b + 
(c_{N}^{T} - c_{B}^{T} {\hspace 0.5mm} A_{B}^{-1} {\hspace 0.5mm} A_{N}){\hspace 0.5mm} x_{N}
\end{align}
$$

To search for a *feasible solution*, pivot iterations are performed to the above dictionary. Only can an iteration be feasible when a variable $x_{j}\in x_{N}$ is able to enter the *basis* $B$ in the new *dictionary* with a corresponding variable $x_{i}\in x_{B}$ to exit. Otherwise, special cases will be performed. The following determines the entering variable:

$$\boldsymbol{x_{j}} = max\{c_{N}^{T} - c_{B}^{T} {\hspace 0.5mm} A_{B}^{-1} {\hspace 0.5mm} A_{N} > 0 \}$$

Likewise, the exiting variable is determined by:
$$
{\hspace 20mm}\boldsymbol{x_{i}} = max\{t \geq 0 : A_{B}^{-1}{\hspace 0.5mm} b - t{\hspace 0.5mm}(A_{B}^{-1}{\hspace 0.5mm}A_{j}) \geq 0 \},
{\hspace 5mm}
\text{where i are rows and j are columns in}{\hspace 1.5mm}A_{ij}
$$

In this algorithm, the *optimal solution* $x_{B} = A_{B}^{-1}{\hspace 0.5mm} b$ is found when $(c_{N}^{T} - c_{B}^{T} {\hspace 0.5mm} A_{B}^{-1} {\hspace 0.5mm} A_{N}) \leq 0$. Thus, the *maximum* value of $z$ is attained at the found optimal solution:

$$
\boldsymbol{z} = c_{B}^{T} {\hspace 0.5mm} A_{B}^{-1} {\hspace 0.5mm} b
$$

It is also important to note that a linear programming problem may ask to *minimize* the objective function instead of maximizing it as shown in the graphical representation below.

![linear programming](http://www.personal.psu.edu/cxg286/Images/MyAnimatedGif.gif)

### Special Cases of L.P. Problems

$\underline{\text{Multiple Optimal Solutions}}$

There is more than one solution to an L.P. problem due to the existence of a free variable provided the problem is bounded. This occurs when there is a $0$ coefficient in the *optimized* objective function: 

$$\exists{\hspace 1mm}0 \in \{c_{N}^{T} - c_{B}^{T} {\hspace 0.5mm} A_{B}^{-1} {\hspace 0.5mm} A_{N} \leq 0 \}$$
<img src="http://4.bp.blogspot.com/-gLQ0UF9ngdc/UHQ7VLMvnBI/AAAAAAAAAOY/3YprIJICB1c/s1600/multopt.png" width="230" height="200" alt="Drawing" align="center"> 

$\underline{\text{Unbounded}}$

There is an infinite number of solutions to an L.P. problem. This occurs when no variable $x_{i}$ is able to exit: 

$$A_{B}^{-1}{\hspace 0.5mm}A_{j}\leq 0$$
<img src="http://4.bp.blogspot.com/_vlUDzVnetcs/TFHVEZuX4gI/AAAAAAAAABY/WhW8NDU62qQ/s1600/extremerays.png" width="260" height="275" alt="Drawing" align="center"> 

$\underline{\text{Infeasible}}$

A special pivot to feasibility is necessarily performed before it can be solved. It goes beyond the scope of this notebook. This occurs when no feasible solutions are able to satisfy the linear constraints $x_{B}$ in the L.P. problem:

$$x_{B} = A_{B}^{-1}{\hspace 0.5mm} b \leq 0$$
<img src="http://www.dataenthusiast.com/wp-content/uploads/2013/02/feasible-900x773.png" width="240" height="300" alt="Drawing" align="center"> 

### Function Definition to Solve Feasible L.P. Problems

Note that the definitions used to formulate the dictionaries in the iterations of an L.P. problem is extracted from the augmented matrix $\tilde{A}_{\Lambda}$ and augmented $\tilde{\bar{c}}_{\Lambda}$

$$
\boldsymbol{\tilde{A}_{\Lambda}} = 
\left(
\begin{array}{cccc|cccc}
a_{1,1} & a_{1,2} & \cdots & a_{1,n} & 1_{1,n+1} & 0 & \cdots & 0\\
a_{2,1} & a_{2,2} & \cdots & a_{2,n} & 0 & \ddots & \cdots & \vdots \\
\vdots & \vdots & \ddots & \vdots & \vdots & \vdots & \ddots & \vdots \\
a_{m,1} & a_{m,2} & \cdots & a_{m,n} & 0 & 0 & \cdots & 1_{m,n+m} \\
\end{array}
\right)
{\hspace 10mm}
\boldsymbol{\tilde{\bar{c}}_{\Lambda}} = 
\begin{pmatrix}
c_{1}\\c_{2} \\ \vdots \\c_{n} \\  \hline 0 \\ \vdots \\ {\hspace 6.5mm}
0_{n+m}
\end{pmatrix}
$$
based on the sets:
$$
\boldsymbol{N} = \{ 1, 2, \ldots, n\}
{\hspace 10mm}
\boldsymbol{B} = \{ n+1, n+2, \ldots, n+m\}
$$
where an initial dictionary has:
$$
\boldsymbol{A_{N}} = 
\begin{pmatrix}
a_{11} & a_{12} & \cdots & a_{1n} \\
a_{21} & a_{22} & \cdots & a_{2n} \\
\vdots & \vdots & \ddots & \vdots \\
a_{m1} & a_{m2} & \cdots & a_{mn}
\end{pmatrix}
{\hspace 10mm}
\boldsymbol{A_{B}} = 
\begin{pmatrix}
1_{1,n+1} & 0 & \cdots & 0 \\
0 & \ddots & \cdots & \vdots \\
\vdots & \vdots & \ddots & \vdots \\
0 & 0 & \cdots & 1_{m,n+m}
\end{pmatrix}
{\hspace 10mm}
\boldsymbol{\bar{c}_{N}} = 
\begin{pmatrix}
c_{1}\\c_{2} \\ \vdots \\c_{n}
\end{pmatrix}
{\hspace 10mm}
\boldsymbol{\bar{c}_{B}} = 
\begin{pmatrix}
0\\ 0 \\ \vdots \\ \quad {\hspace 1.4mm} 0_{n+m}
\end{pmatrix}
$$

with the columns of $A_{B}$ and rows of $c_{B}$ representing the [slack variables](http://pblpathways.com/fm/C4_3_2.pdf) ${\hspace 3mm}\{x_{n+1},\ldots,x_{n+m}\}{\hspace 3mm}$ and the columns of $A_{N}$ and rows of $c_{N}$ representing the [decision variables](https://www.courses.psu.edu/for/for466w_mem14/Ch11/HTML/Sec1/ch11sec1_Vars.htm) ${\hspace 3mm}\{x_{1},\ldots,x_{n}\}{\hspace 3mm}$.

Therefore, the main idea of this algorithm is that the elements of $B$ and $N$ are inter-changed to each other until the definitions above with the iterated $B$ and $N$ formulate the optimal solution. Hence, we can write our own function to solve feasible linear programming problems:  

In [ ]:
def lp(c,A,b,max_iter=10000):
    """Takes:
    
    A = a list of length m (containing sublists of length n);
    b = a list of length m; 
    c = a list of length n;
    max_iter = maximum number of iterations, optional; 
    
    Returns the maximum value of the objective function and the optimal solution it can be attained at"""
    m         = len(b)
    n         = len(c)
    b         = np.array(b)
    A         = np.array(A)
    I         = np.identity(m)
    slacks    = np.zeros(m)
    decisions = np.array(c)
    
    # This is the augmented matrix A and the augmented vector c
    A_augment = np.concatenate((A,I),axis=1) 
    c_augment = np.concatenate((decisions,slacks),axis=0)
    
    
    # Initial Dictionary with c_B having slack variables in the basis, and c_N having the decision variables
    A_B     = A_augment[:,-m:]
    A_B_inv = np.linalg.inv(A_B)
    A_N     = A_augment[:,0:-m:]
    c_B     = c_augment[-m:].reshape(1,m)       # reshape because in the formula above c_B is transposed
    c_N     = c_augment[0:-m:].reshape(1,n)     # reshape because in the formula above c_N is transposed
    
    # To serve as condition for the while loop
    enter = c_N - (c_B @ A_B_inv @ A_N)
    
    # This is a list to keep count on the number of iterations
    p = []
    iterations = len(p)
    
    # This is to to be able to compute the optimal solutions given the input L.P. problem is already optimized
    # Descriptions of what the following means are further down below
    j   = np.argmax(enter)
    A_j = A_augment[:,j].reshape(m,1)
    d   = A_B_inv @ A_j
    
    # L.P. Problem is not optimized, hence, iterate inside while loop
    while np.any(enter > 0):     # if enter <= 0 then optimal solution is found
        if np.any(b < 0):
            return "L.P. Problem is INFEASIBLE"
        else:
            # To enter the pivot iteration (while loop)
            enter = c_N - (c_B @ A_B_inv @ A_N)
            enter = enter.reshape(n)                            # reshape to be able to perform operations to it
            
            # To determine the entering variable                    
            j   = np.argmax(enter)                              # j is the index of the entering variable
            c_j = c_N[:,j]
            A_j = A_augment[:,j].reshape(m,1)                   # this is the entering column (variable) to A_B
            
            # To determine the exiting variable
            x_B = (A_B_inv @ b).reshape(m,1)
            d   = A_B_inv @ A_j
            if np.all(d <= 0):
                return "L.P. Problem is UNBOUNDED"
            else:
                t_list = []
                # t is the multiple of the coefficients in x_j
                for t in np.linspace(0,np.amax(x_B),1000):      # range is [0,max{x_B}] since {x_B - td >= 0}      
                    if np.all(x_B - (t*d) >= 0):
                        t_list.append(t)                      
                # This is to satisfy the condition max{t >= 0: x_b - t*A_B_inv*A_N} >= 0}
                t     = np.amax(np.array(t_list))
                i     = np.argmin(x_B - (t*d))              # i is the index of the exiting variable
                A_i   = A_B[:,i]                            # this is the exiting column (variable) from A_B
                c_i   = c_B[:,i]                            # this is the exiting column (variable) from c_B
                
                # To avoid modifying the A_augment matrix outside the while loop
                A_N = np.copy(A_N)
                A_B = np.copy(A_B)
                c_B = np.copy(c_B)
                c_N = np.copy(c_N)
                
                # To define the new set of dictionary
                A_N[:,j] = A_i
                A_B[:,i] = A_j.reshape(m)
                c_N[:,j] = c_i
                c_B[:,i] = c_j
                
                # To control the definitions we modified just above this section
                A_B = A_B[:]
                A_N = A_N[:]
                c_B = c_B[:]
                c_N = c_N[:]
                A_B_inv  = np.linalg.inv(A_B)
            
                # To test whether to exit or continue the iteration (while loop)
                enter = c_N - (c_B @ A_B_inv @ A_N)
                
                # To keep count on the number of iterations done
                p.append(1)
                iterations = len(p)
                if iterations > max_iter:
                    x_B = A_B_inv @ b
                    z = c_B @ A_B_inv @ b
                    return "Warning: Maximum number of iterations exceeded.","Maximum value = " + str(tuple(z)), "attained at" + str(tuple(x_B)) + " in the current iteration"     
    # Handles multiple optimal solutions up to 1 free variable only 
    if np.any(enter == 0):
        # The optimal solution(s)
        x_B = A_B_inv @ b
        z = c_B @ A_B_inv @ b
        
        # This code cannot handle more than 1 free variable
        if np.array(np.where(enter == 0))[0].size > 1:
            return "Warning: Optimal solution has more than 1 free variable. This code is not designed for multiple optimal solutions with 2 or more free variables. It may cause errors in the future."
        
        # This is a very special case where there is still a unique optimal solution despite a free variable
        elif np.any(x_B == 0) and d[int(np.array(np.where(x_B == 0))[-1])] > 0:
            return "Maximum value = " + str(tuple(z)), "attained at" + str(tuple(x_B))
        
        # Otherwise, it has multiple optimal solutions
        else:
            # To display x_B with the free variable 't'
            the_list = []
            for ii,jj in enumerate(x_B):
                for iii,jjj in enumerate(d):
                    if ii == iii:
                        the_list.append(str(jj) + " + " + str(int(jjj)) + "t")           
            return "Maximum value = " + str(tuple(z)), "attained at" + str(the_list) + "  for 0 ≤ t ≤ " + str(t)
    # A unique optimal solution is found
    else:
        # The optimal solution
        x_B = A_B_inv @ b
        z = c_B @ A_B_inv @ b
        return "Maximum value = " + str(tuple(z)), "attained at" + str(tuple(x_B))

### `scipy.optimize.linprog`

The function `linprog` **minimizes** a linear objective function ${\hspace 2mm}z \leq \bar{c}^{\intercal}{\hspace 0mm}x{\hspace 2mm}$subject to both linear equality and inequality constraints ${\hspace 2mm}\boldsymbol{A}{\hspace 0.5mm}x \leq \bar{\boldsymbol{b}}{\hspace 2mm}$. It takes at least an array-like input `c` which represents the coefficients of the linear objective function $z$. This parameter is the only *required* input of `linprog` since an L.P. problem can have an objective function with no constraints at all. The array-like inputs are of types such as list or array.  

However, there a few important optional inputs that `linprog` can take. It notably distinguishes a linear constraint with equality $\{{\hspace 0.25mm}={\hspace 0.25mm}\}$ or inequality $\{{\hspace 0.25mm}\leq{\hspace 0.25mm},{\hspace 0.25mm}\geq{\hspace 0.25mm}\}$. The inputs `A_ub` and `b_ub` take elements of the matrix $A$ and vector $b$ correspondingly to the rows of **inequality**${\hspace 2mm}\boldsymbol{A}{\hspace 0.5mm}x \leq \bar{\boldsymbol{b}}{\hspace 2mm}$. Meanwhile, the inputs `A_eq` and `b_eq` take elements of the matrix $A$ and vector $b$ correspondingly to the rows of **equality**${\hspace 2mm}\boldsymbol{A}{\hspace 0.5mm}x = \bar{\boldsymbol{b}}{\hspace 2mm}$. Hence, the linear programming problem do not necessarily have to be in standard form for `linprog` to solve it. However, it is highly important to note that `linprog` only accepts the above optional inputs if the lengths of `b_ub` and `b_eq` are equal to the rows of `A_ub` and `A_eq` and the columns of `A_ub` and `A_eq` are equal to the length of `c` respectively. Otherwise, it will give an `error` message. For references of this row and column parity, go back to the L.P. model that has been defined above.

Another important optional input `linprog` can take is `bounds`. It is the pair `(min,max)` that defines the range of values that each element in $x$ can be defined. By default, `bounds` are `(0,None)` - i.e. the positivity constraint $\forall{x}\geq{0}$. Meanwhile, the rest of the optional inputs are trivial to the end process of reaching optimality, but more information is provided in the function's [documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linprog.html): `scipy.optimize.linprog?`

Lastly, the output of the `linprog` function returns 7 specific entries:
* $fun$ ${\hspace 10mm}$$\rightarrow$ the value of the objective function
* $message$${\hspace 2.5mm}$$\rightarrow$ the text information about the status of the optimization respective of the `integer` of $status$ described below
* $nit$ ${\hspace 11.775mm}$$\rightarrow$ the number of iterations performed 
* $slack$${\hspace 8.35mm}$$\rightarrow$ the values of the slack variables at optimality 
* $status$${\hspace 7.1mm}$$\rightarrow$ an integer from the list `[ 0, 1, 2, 3 ]`; please see [documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linprog.html) to go in full details as to what they represent
* $success$${\hspace 4.55mm}$$\rightarrow$ True/False whether optimality was reached
* $x$${\hspace 16mm}$$\rightarrow$ the optimal solution

## 2. The Standard Maximum (Minimum) Problem

Let us solve the following simple linear programming problem:
$$
Max{\hspace 5mm} 5x_{1} + 6x_{2} + 9x_{3} + 8x_{4} \\
\text{subject to}
\begin{cases}
x_{1} + 2x_{2} + 3x_{3} + x_{4} \leq 5\\
x_{1} + x_{2} + 2x_{3} + 3x_{4} \leq 3\\
\hline
x_{1},x_{2},x_{3},x_{4} \geq 0
\end{cases}
$$

Firstly, we define the only requirement of `scipy.optimize.linprog` function `c`. In this case, $c = [5,6,9,8]$.

In [ ]:
c = [5,6,9,8]

To make a point of a prior claim, we can try and solve the L.P. problem with only the `c` input having no linear constraints with the `scipy.optimize.linprog` function:

In [ ]:
linprog(c)

As you can see, the L.P. problem terminates successfully, but finds a trivial optimal solution which really means nothing other than a null set of solutions.

Moving on, we define the constraints: matrix A as a `list` with each element as sublists representing the **rows**, while the items inside the `sublists` representing the **columns** of the matrix $A$:
$$
\boldsymbol{A} = 
\begin{bmatrix}
1 & 2 & 3 & 1 \\
1 & 1 & 2 & 3 
\end{bmatrix}
$$

In [ ]:
A = [[1,2,3,1],[1,1,2,3]]

Then, we define the upper limits of our linear **inequality** constraints: ${\hspace 20mm}\boldsymbol{A}{\hspace 0.5mm}x \leq \bar{\boldsymbol{b}}{\hspace 2mm}$

In [ ]:
b = [5,3]

Since we do not have any linear **equality** constraints and our `bounds` are of the default values, we will now continue in actually solving the L.P. problem with the function:

In [ ]:
linprog(c, A_ub=A, b_ub=b) 

Woops! That does not seem correct... How can we tell that the returned output is not correct? Well, our linear programming problem is a maximization and, therefore, we should be expecting the value of the objective function $z$ to be increasing as explained graphically in the *video* below. If it is not increasing then either the L.P. problem is **degenerate** - in which the objective function $z$ does not increase; or an `error` has been made somewhere. Hence, I claim that an error has been made.

In [ ]:
from IPython.display import YouTubeVideo
from datetime import timedelta

start=int(timedelta(minutes=4, seconds=58).total_seconds())
end=int(timedelta(minutes=5, seconds=44).total_seconds())

YouTubeVideo("87OKtTpSRB8", start=start, end=end, autoplay=1, theme="light", color="blue")

Also, we have to remember that `scipy.optimize.linprog` is a minimization function and the problem we have stated above is a maximization problem. Therefore, we have to convert it to a minimization problem for us to make use of the `linprog` function! It is very easy to do:

$$
\begin{matrix}
Max \qquad \quad 5x_{1} + 6x_{2} + 9x_{3} + 8x_{4} \\
\text{subject to}
\begin{cases}
x_{1} + 2x_{2} + 3x_{3} + x_{4} \leq 5\\
x_{1} + x_{2} + 2x_{3} + 3x_{4} \leq 3\\
\hline
x_{1},x_{2},x_{3},x_{4} \geq 0
\end{cases}
\end{matrix}
\qquad
\begin{matrix}
{\rightarrow} & {\rightarrow}\\ {\rightarrow} & {\rightarrow}\\ {\rightarrow} & {\rightarrow}
\end{matrix}
\qquad
\begin{matrix}
Min \qquad -5x_{1} - 6x_{2} - 9x_{3} - 8x_{4} \\
\text{subject to}
\begin{cases}
x_{1} + 2x_{2} + 3x_{3} + x_{4} \leq 5\\
x_{1} + x_{2} + 2x_{3} + 3x_{4} \leq 3\\
\hline
x_{1},x_{2},x_{3},x_{4} \geq 0
\end{cases}
\end{matrix}
$$

In addition, the `linprog` function can take the `A_ub` and `b_ub` input parameters, which signify the "upper bounds". Therefore, the  L.P. problem does not necessarily have to be in the standard form of minimization where lower bounds exist. Hence, the matrix $A$ and vector $b$ remain unchanged, while modifying the only difference of $-c$. So let us define this `c_min` vector.

In [ ]:
c_min = [-5,-6,-9,-8]

Now we can hopefully solve the L.P. problem stated above with the `scipy.optimize.linprog` function:

In [ ]:
linprog(c_min, A_ub=A, b_ub=b) 

Hurray! Therefore, we can conclude that the error we had before was in our $c$, the coefficients of the objective function.

Furthermore, in our minimization we get $z = -17$, however, the L.P. problem above is asking us to maximize the objective function $z$. Hence, we can convert it back to the maximizing problem to address the main task of the problem and we get $z = 17$ attained at $[1,2,0,0]$.

Finally, let us confirm it with our very own function `lp` we defined above:

In [ ]:
lp(c,A,b)

Both results are exactly the same. The `scipy.optimize.linprog` function approaches the $z$ value by **minimizing** from above, hence, the negative value. Meanwhile our very own function `lp` approaches the $z$ value by **maximizing** from the bottom, alas, the positive value.

However, sometimes an L.P. problem do not have a unique optimal solution. It can have multiple optimal solution as shown by a prior graph. Here is a good example of this case:
$$
\mkern-65mu
Maximize \quad 3x_{1} \quad + \quad 2x_{2} \\
\begin{align}
\text{subject to}
\begin{cases}
{\hspace 2mm}x_{1} & &\leq 40\\
& \qquad x_{2} &\leq 60\\
3x_{1} &+  \quad 2x_{2}  &\leq 180\\
\hline
& \quad x_{1},x_{2} &\geq 0
\end{cases}
\end{align}
$$

In [ ]:
A1 = [[1,0],[0,1],[3,2]]
c1 = [3,2]
b1 = [40,60,180]

We have defined the necessary inputs for the function `linprog`, so let us try and solve it.

In [ ]:
linprog(c1,A_ub=A1,b_ub=b1)

Woops! We forgot that `linprog` is a minimization function, once again! Therefore, we have to change it to a minimization problem with upper bounds.

In [ ]:
c1_min = [-3,-2]

In [ ]:
linprog(c1_min,A_ub=A1,b_ub=b1)

That seems about right since our $z = 180$ and, therefore, objective function is increasing if maximized. For further assurance, let us confirm it with our very own function.

In [ ]:
lp(c1,A1,b1)

Perfect! They are almost identical and as you can see our very own defined function `lp` is even more specific about the optimality as it tells you that there are more than one optimal solution since `t` can vary within a bounded region.

Now that we are at the end of this notebook you should be able to solve feasible linear programming problems with the `scipy.optimize.linprog` function with no trouble at all. Evidently, we have gone through a very common error in using this function - that is forgetting to convert it to a minimization problem with upper bounds or equality constraints and, most importantly, $-c$. 

On a side note, even though this notebook did not cover any *infeasible* linear programming problems, those types of L.P. problems are quite easily solved with the `linprog` function when a [special pivot to feasibility](http://lpsolve.sourceforge.net/5.5/Infeasible.htm) has been performed to it. If interested, you can watch the video below at full length for a thorough demonstration in doing so. Please take note that the concept in the video is explained in [tableau form](https://www.utdallas.edu/~scniu/OPRE-6201/documents/LP06-Simplex-Tableau.pdf), which is an alternative to a [dictionary form](http://cis.poly.edu/POG/canon.pdf) - that of the latter is the format of the algorithm in this notebook. However, they have the same information and the techniques used to find optimality are of no differences.

In [ ]:
from IPython.display import YouTubeVideo
from datetime import timedelta

end=int(timedelta(minutes=11, seconds=9).total_seconds())

YouTubeVideo("jFWL3d6x5lA", end=end, autoplay=1, theme="light", color="blue")

If you watched the video until it automatically stops, it should have brought you to a point where an **infeasible** L.P. problem has been converted to a **feasible** L.P. problem. Thus, the `scipy.optimize.linprog` function can be used to solve it from that point on. Now try and define the matrix $A$, the coefficient vector $c$, and the upper bounds $b$ from the tableau for yourself and compare what you get with the solution in the video!

If you got that easily, good for you! Have a try and do more challenging exercises below which should complement to your understanding of the function `scipy.optimize.linprog`:

## 3. Exercises

**Exercise 3.1** 

(a) Define the matrix $A$, the coefficient vector $c$, and the upper bounds vector $b$ as lists in the following L.P. problem: 

(b) Find the optimal solution(s) of the problem using the `linprog` function and determine what the *maximum* and * minimum* values are that the objective function can attain.

$$
{\hspace -15mm}Maximize{\hspace 5mm} 3x_{1} + 2x_{2} \\
\text{subject to}
\begin{align}
\begin{cases}
x_{1} - 2x_{2} &\leq 1\\
x_{1} - x_{2} &\leq 2\\
2x_{1} - x_{2} &\leq 6\\
x_{1}  &\leq 5\\
2x_{1} + x_{2} &\leq 16\\
x_{1} + x_{2} &\leq 12\\
x_{1} + 2x_{2} &\leq 21\\
\qquad x_{2} &\leq 10\\
\hline
\quad x_{1},x_{2} &\geq 0
\end{cases}
\end{align}
$$

**Exercise 3.2** 

(a) Convert the following L.P. problem to the standard form.

(b) Define the matrix $A$, the coefficient vector $c$, and the upper bounds vector $b$

(c) Find the optimal solution(s) that can attain the *maximum* value of the objective function using `linprog`.

$$
\mkern-150mu
Minimize \quad \qquad 5 + x_{1} + 2x_{2} + 3x_{3} + 4x_{4} \\
\begin{align}
\text{subject to}
\begin{cases}
\mkern-70mu \qquad \qquad \qquad 4x_{1} &+ \quad 3x_{2} &+ \quad 2x_{3} &+ \quad x_{4} &\leq 10\\
\mkern-70mu \qquad \qquad \qquad x_{1}  &         &- \quad x_{3} &+ \quad 2x_{4} &= 2\\
\mkern-70mu \qquad \qquad \qquad x_{1} &+ \quad x_{2} &+ \quad x_{3} &+ \quad x_{4} &\geq 1\\
\hline
&&&x_{1},x_{2},x_{3},x_{4} &\geq 0
\end{cases}
\end{align}
$$

* $\underline{\text{Hint}}$: `linprog(A_eq,b_eq)`

**Exercise 3.3**

A small company manufactures two electronic components, plastic cases for USB drives with two ports and cases with one port. Each two-port case takes `2` hours to assemble, whereas a one port case requires `1` hour. There are a maximum of `50` hours of labor dedicated for the day. The cost of materials for two-port case is `$1`; the material for a one-port case costs `$2`. Production only allows a maximum of `$70` per day. Each two-case port earns a proﬁt of `$4` and each one-port case `$5`.

(a) Set up the linear programming model to the above problem. Define the matrix $A$, the coefficient vector $c$, and the upper bounds vector $b$.

(b) Find the number of two-port cases and one-port cases produced that will maximize the proﬁt using the `linprog` function.